In [ ]:
1. Collect get database for politicians
	- Start with members of the current goverment
	- Continue with the whole Bundestag

2. Write a function to collect tweet data for a specific user id for a specific month in a Pandas dataframe

3. Build a database to store x data

3. Write a function to continue an ongoing collection process for a given user id



4. Write a funciton to collect answer tweets to a specific tweet id

5. Write a function to continue answer collection for a specific tweet id

In [2]:
from lxml import etree
import pandas as pd

In [8]:
# Parse XML
tree = etree.parse("data/MDB_STAMMDATEN.XML")
root = tree.getroot()

records = []

for mdb in root.findall(".//MDB"):
    # Step 1: Check if active (any WAHLPERIODE with no MDBWP_BIS)
    wps = mdb.find("WAHLPERIODEN")
    if wps is None:
        continue

    active_wp = None
    for wp in wps.findall("WAHLPERIODE"):
        bis = wp.findtext("MDBWP_BIS")
        if bis is None or bis.strip() == "":
            active_wp = wp
            break

    if active_wp is None:
        continue  # not active

    record = {}

    # --- Basic ID ---
    record["ID"] = mdb.findtext("ID")

    # --- Biographic Info ---
    bio = mdb.find("BIOGRAFISCHE_ANGABEN")
    if bio is not None:
        for child in bio:
            record[child.tag] = child.text

    # --- Name Info ---
    name = mdb.find(".//NAMEN/NAME")
    if name is not None:
        for part in name:
            record[part.tag] = part.text

    # --- Active Wahlperiode Info ---
    if active_wp is not None:
        for wp_field in active_wp:
            record[f"WP_{wp_field.tag}"] = wp_field.text

    records.append(record)

# Final DataFrame
df = pd.DataFrame(records)
print(df.head())
print(f"Total active MDBs: {len(df)}")


         ID GEBURTSDATUM GEBURTSORT  GEBURTSLAND STERBEDATUM GESCHLECHT  \
0  11000756   16.01.1948     Berlin  Deutschland        None   männlich   
1  11002718   18.02.1961     Aachen  Deutschland        None   männlich   
2  11002720   19.01.1968     Dessau  Deutschland        None   weiblich   
3  11002733   09.06.1961     Lorsch  Deutschland        None   männlich   
4  11002735   11.11.1955     Brilon  Deutschland        None   männlich   

           FAMILIENSTAND            RELIGION                    BERUF  \
0   geschieden, 3 Kinder      konfessionslos             Rechtsanwalt   
1  verheiratet, 3 Kinder  römisch-katholisch  Ministerpräsident a. D.   
2     geschieden, 1 Kind                None   Dipl.-Agraringenieurin   
3  verheiratet, 2 Kinder         evangelisch       Dipl.-Mathematiker   
4  verheiratet, 3 Kinder  römisch-katholisch             Rechtsanwalt   

             PARTEI_KURZ  ... HISTORIE_BIS WP_WP WP_MDBWP_VON WP_MDBWP_BIS  \
0             DIE LINKE.  ...   

In [9]:
df["VITA_KURZ"] = df["VITA_KURZ"].str.replace(";", "")
df["VITA_KURZ"] = df["VITA_KURZ"].str.replace(":", "")
df["VITA_KURZ"] = df["VITA_KURZ"].str.replace(",", "")
df["VITA_KURZ"] = df["VITA_KURZ"].str.replace(".", "-")
df["WP_INSTITUTIONEN"] = df["WP_INSTITUTIONEN"].str.replace("\n", "")

In [14]:
df['ID'] = df['ID'].astype(str)

In [ ]:
df2 = pd.read_excel("data/politicians.xlsx", sheet_name="Table1")

In [17]:
df2['ID'] = df2['ID'].astype(str)

In [18]:
df2['ID'].dtype, df['ID'].dtype

(dtype('O'), dtype('O'))

In [19]:
df1 = df.merge(df2[['ID', 'USERNAME']], on='ID')

In [23]:
df1[df1['ID'].isin(["11002735", "11004182"])]

,ID,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,BERUF,PARTEI_KURZ,...,WP_WP,WP_MDBWP_VON,WP_MDBWP_BIS,WP_WKR_NUMMER,WP_WKR_NAME,WP_WKR_LAND,WP_LISTE,WP_MANDATSART,WP_INSTITUTIONEN,USERNAME
4,11002735,11.11.1955,Brilon,Deutschland,None,männlich,"verheiratet, 3 Kinder",römisch-katholisch,Rechtsanwalt,CDU,...,21,25.03.2025,None,146,Hochsauerlandkreis,NW,NW,Direktwahl,,_FriedrichMerz
5,11002735,11.11.1955,Brilon,Deutschland,None,männlich,"verheiratet, 3 Kinder",römisch-katholisch,Rechtsanwalt,CDU,...,21,25.03.2025,None,146,Hochsauerlandkreis,NW,NW,Direktwahl,,bundeskanzler
88,11004182,10.02.1963,Husum,Deutschland,None,männlich,"verheiratet, 3 Kinder",evangelisch,Rechtsanwalt,CDU,...,21,25.03.2025,None,4,Rendsburg-Eckernförde,SH,SH,Direktwahl,,JoWadephul
89,11004182,10.02.1963,Husum,Deutschland,None,männlich,"verheiratet, 3 Kinder",evangelisch,Rechtsanwalt,CDU,...,21,25.03.2025,None,4,Rendsburg-Eckernförde,SH,SH,Direktwahl,,AussenMinDE


In [24]:
df1.to_excel('data/politicians3.xlsx', index=False)

In [9]:
df.to_csv('data/politicians.csv', index=False)

In [ ]:
df.to_excel('data/politicians3.xlsx', index=False)

In [3]:
df1 = pd.read_excel("data/politicians2.xlsx", sheet_name="Sheet1")
df2 = pd.read_excel("data/politicians.xlsx", sheet_name="Table1")

In [4]:
df1_only = df1[~df1['ID'].isin(df2['ID'])]
df2_only = df2[~df2['ID'].isin(df1['ID'])]


In [5]:
df1_only

,ID,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,BERUF,PARTEI_KURZ,...,HISTORIE_BIS,WP_WP,WP_MDBWP_VON,WP_MDBWP_BIS,WP_WKR_NUMMER,WP_WKR_NAME,WP_WKR_LAND,WP_LISTE,WP_MANDATSART,WP_INSTITUTIONEN


In [7]:
df2_only['NACHNAME']

6    Rachel
Name: NACHNAME, dtype: object

In [22]:
import tweepy

# Initialize the client
client = tweepy.Client(bearer_token=bearer_token)

# Replace with the tweet ID you want to inspect
tweet_id = 1943616979077648534

# Request the tweet with public metrics
tweet = client.get_tweet(
    id=tweet_id,
    tweet_fields=["public_metrics", "created_at", "lang", "source"]
)

# Access the tweet data
data = tweet.data

print("Tweet Text:", data.text)
print("Likes:", data.public_metrics["like_count"])
print("Retweets:", data.public_metrics["retweet_count"])
print("Replies:", data.public_metrics["reply_count"])
print("Quotes:", data.public_metrics["quote_count"])
print("Created at:", data.created_at)
print("Language:", data.lang)
print("Source:", data.source)


Tweet Text: Verantwortung für Deutschland: das ist das Leitmotiv der Bundesregierung. Die Länder ziehen mit: Der Bundesrat hat dem Investitionsbooster zugestimmt, der ersten großen Reform der Unternehmensbesteuerung seit 15 Jahren. Damit wird unser Land wieder attraktiv und wettbewerbsfähig. https://t.co/qqbjL41sHy
Likes: 657
Retweets: 90
Replies: 1242
Quotes: 36
Created at: 2025-07-11 10:22:53+00:00
Language: de
Source: None
